In [6]:
# Importamos librerias
import findspark
findspark.init()

import pyspark
from pyspark.sql.functions import *

from pyspark.sql import SparkSession

#Creamos la session
spark = (SparkSession
        .builder
        .appName("Nasa")
        .getOrCreate())

In [10]:
# Cargamos el archivo parquet con el que vamos a trabajar

nasa = spark.read.parquet("Nasa.parquet")

In [11]:
nasa.show()

+--------------------+------+--------------------+--------+------+-----+-------------------+
|                host|method|           resources|protocol|status| size|               time|
+--------------------+------+--------------------+--------+------+-----+-------------------+
|        199.72.81.55|   GET|    /history/apollo/|HTTP/1.0|   200| 6245|1995-07-01 00:00:01|
|unicomp6.unicomp.net|   GET| /shuttle/countdown/|HTTP/1.0|   200| 3985|1995-07-01 00:00:06|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200| 4085|1995-07-01 00:00:09|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   304|    0|1995-07-01 00:00:11|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200| 4179|1995-07-01 00:00:11|
|  burger.letters.com|   GET|/images/NASA-logo...|HTTP/1.0|   304|    0|1995-07-01 00:00:12|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   200|    0|1995-07-01 00:00:12|
|     205.212.115.106|   GET|/shuttle/countdow...|HTTP/1.0|   200| 398

In [14]:
# Vemos cuantos registros tenemos en el df
nasa.count()

3461612

In [20]:
#¿Cuáles son los protocolos web más utilizados? Agrupelos
protocolos = (nasa.groupBy('protocol')
                  .count()
                  .sort('count', ascending = False))

protocolos.show()

+-------------+-------+
|     protocol|  count|
+-------------+-------+
|     HTTP/1.0|3454716|
|             |   6599|
|    HTTP/V1.0|    279|
|       HTTP/*|     13|
|STS-69</a><p>|      4|
|            a|      1|
+-------------+-------+



In [29]:
#Observamos que hay protocolos con cadenas vacias, al ser una gran cantidad de campos vacios
# los reemplazaremos por null

null_protocol = nasa.na.replace('', None)

In [34]:
# Volvemos a formular la pregunta con los campos corregidos
nasa_df1 = (null_protocol.groupBy('protocol')
            .count()
            .sort('count', ascending = False))

nasa_df1.show()

+-------------+-------+
|     protocol|  count|
+-------------+-------+
|     HTTP/1.0|3454716|
|         null|   6599|
|    HTTP/V1.0|    279|
|       HTTP/*|     13|
|STS-69</a><p>|      4|
|            a|      1|
+-------------+-------+



In [35]:
# Cuáles son los códigos de estado mas comunes en la web? Agrupelos
status_code = (null_protocol.groupBy('status')
              .count()
              .sort('count', ascending = False))

status_code.show()

+------+-------+
|status|  count|
+------+-------+
|   200|3100524|
|   304| 266773|
|   302|  73070|
|   404|  20899|
|   403|    225|
|   500|     65|
|   501|     41|
|   400|     15|
+------+-------+



In [36]:
# ¿los métodos de petición mas utilizados?

methods = (null_protocol.groupBy('method')
          .count()
          .sort('count', ascending = False))
methods.show()

+---------------+-------+
|         method|  count|
+---------------+-------+
|            GET|3451720|
|           HEAD|   7915|
|           null|   1753|
|           POST|    222|
|�|t�9ð'À|u|      2|
+---------------+-------+



In [37]:
# Vemos que hay métodos con cadenas vacias, anteriormente aplicamos a las cadenas vacias la sutitución por null.
# Hay otro método con error en el nombre, con 2 valores. Habria que estudiarlo si eliminarlo o ver de cual se trata.

In [43]:
# ¿Qué recurso tuvo la mayor transferencia de bytes de la página web?
nasa_complete = null_protocol

nasa_complete.orderBy(desc('size')).limit(1).show()

+----+------+--------------------+--------+------+-------+-------------------+
|host|method|           resources|protocol|status|   size|               time|
+----+------+--------------------+--------+------+-------+-------------------+
|null|   GET|/shuttle/countdow...|HTTP/1.0|   200|6823936|1995-07-07 14:03:32|
+----+------+--------------------+--------+------+-------+-------------------+



In [47]:
# Además, queremos saber que recurso de nuestra web es el que más tráfico recibe.
#Es decir, el recurso con más registros en nuestro log.
nasa_complete.groupBy('resources').count().orderBy(desc('count')).limit(1).show()

+--------------------+------+
|           resources| count|
+--------------------+------+
|/images/NASA-logo...|208714|
+--------------------+------+



In [55]:
nasa_complete.groupBy('time').count().orderBy(desc('count')).show()

+-------------------+-----+
|               time|count|
+-------------------+-----+
|1995-07-13 13:32:58|   20|
|1995-08-29 11:11:47|   18|
|1995-07-13 09:42:19|   18|
|1995-07-13 13:32:59|   18|
|1995-07-13 09:42:20|   17|
|1995-08-01 08:18:28|   17|
|1995-07-13 08:35:02|   16|
|1995-08-15 15:00:36|   16|
|1995-08-15 14:56:55|   16|
|1995-07-13 09:49:30|   16|
|1995-07-11 14:47:44|   16|
|1995-07-13 09:34:06|   15|
|1995-07-17 14:20:57|   15|
|1995-08-30 11:18:22|   15|
|1995-07-05 16:29:15|   15|
|1995-07-13 09:27:16|   15|
|1995-07-12 13:39:35|   15|
|1995-08-10 19:20:46|   15|
|1995-07-13 08:56:34|   15|
|1995-07-13 09:51:04|   14|
+-------------------+-----+
only showing top 20 rows



In [56]:
# ¿Cuáles son los host más frecuentes?
hosts = (nasa_complete.groupBy('host')
          .count()
          .sort('count', ascending = False))
hosts.show()

+--------------------+-----+
|                host|count|
+--------------------+-----+
|piweba3y.prodigy.com|21988|
|piweba4y.prodigy.com|16437|
|piweba1y.prodigy.com|12825|
|  edams.ksc.nasa.gov|11964|
|        163.206.89.4| 9697|
|         news.ti.com| 8161|
|www-d1.proxy.aol.com| 8047|
|  alyssa.prodigy.com| 8037|
|                null| 7660|
| siltb10.orl.mmc.com| 7573|
|www-a2.proxy.aol.com| 7516|
|www-b2.proxy.aol.com| 7266|
|piweba2y.prodigy.com| 7246|
|www-b3.proxy.aol.com| 7218|
|www-d4.proxy.aol.com| 7211|
|www-b5.proxy.aol.com| 7080|
|www-d2.proxy.aol.com| 6984|
|www-b4.proxy.aol.com| 6972|
|www-d3.proxy.aol.com| 6895|
|    webgate1.mot.com| 6749|
+--------------------+-----+
only showing top 20 rows



In [58]:
#¿A que horas se produce el mayor número de tráfico en la web?
(nasa_complete.select(hour('time').alias('hour'))
            .groupBy('hour')
            .count()
            .orderBy(desc('count'))).show()

+----+------+
|hour| count|
+----+------+
|  15|230665|
|  12|227228|
|  13|225350|
|  14|223873|
|  16|217564|
|  11|211064|
|  10|193816|
|   9|178664|
|  17|178443|
|   8|149193|
|  18|146091|
|  22|131432|
|  19|131091|
|  21|129907|
|  20|129753|
|  23|123932|
|   0|110312|
|   7|101403|
|   1| 91597|
|   2| 77805|
+----+------+
only showing top 20 rows



In [64]:
# ¿Cuál es el número de errores 404 que ha habido cada dia?
from pyspark.sql import functions as F

error404 = nasa_complete.filter(nasa_complete['status'] == 404)

(error404.groupBy(F.dayofmonth('time').alias('day'))
        .count()
        .sort('day')).show()

+---+-----+
|day|count|
+---+-----+
|  1|  559|
|  2|  291|
|  3|  778|
|  4|  705|
|  5|  733|
|  6| 1013|
|  7| 1107|
|  8|  691|
|  9|  627|
| 10|  713|
| 11|  734|
| 12|  667|
| 13|  748|
| 14|  700|
| 15|  581|
| 16|  516|
| 17|  677|
| 18|  721|
| 19|  848|
| 20|  740|
+---+-----+
only showing top 20 rows

